##This notebook is for basic QC on sequencing data

In [ ]:
# %install_ext https://raw.githubusercontent.com/SchlossLab/ipython-mothurmagic/master/mothurmagic.py
# Only needs to be done once - gets the mothurmagic so you can run mother easily in the %notebook

In [ ]:
%load_ext mothurmagic
# Loads mothurmagic so we can run mothur in the notebook using %%mothur at the top of the cell

In [ ]:
%%mothur
help()

In [ ]:
!echo $PATH
#Make sure mothur folder is in your path somehow. If not, enter 

In [ ]:
!export PATH=$PATH:opt/virt_env/bin/mothur

First we need to cut out sequences with errors

In [ ]:
! usearch -fastq_filter ../../SeqData/pear_merged-2015-07-03.assembled.demult.fastq \
-fastaout ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.fasta \
-fastq_maxee 1
# Using Roger Edgar's max expected error filtering
# Note we needed to remove - dashes from the name for future mothur processing

####If the file were bigger, we would need to split it (below)

In [ ]:
!wc -l ../../SeqData/pear_merged-2015-06-30.assembled.fastq
# Counts the number of lines in the fastq file. They come in groups of 4 lines.

In [ ]:
! usearch -fastq_filter ../../SeqData/pear_merged-2015-06-30.assembled.fastq \
-fastaout ../../SeqData/pear_merged-2015-06-30.assembled.maxee.fasta \
-fastq_maxee 1
# Using Roger Edgar's max expected error filtering - the whole file is too big to do at once so needs to be split

In [ ]:
!head -16528900 ../../SeqData/pear_merged-2015-06-30.assembled.fastq > \
../../SeqData/pear_merged-2015-06-30.assembled.split1.fastq
!tail -16528900 ../../SeqData/pear_merged-2015-06-30.assembled.fastq > \
../../SeqData/pear_merged-2015-06-30.assembled.split2.fastq
# Split the data for usearch into two files, divided by the number of lines it has as determined above

In [ ]:
! usearch -fastq_filter ../../SeqData/pear_merged-2015-06-30.assembled.split1.fastq \
-fastaout ../../SeqData/pear_merged-2015-06-30.assembled.split1.maxee.fasta \
-fastq_maxee 1
# Using Roger Edgar's max expected error filtering on our split fastq file

In [ ]:
! usearch -fastq_filter ../../SeqData/pear_merged-2015-06-30.assembled.split2.fastq \
-fastaout ../../SeqData/pear_merged-2015-06-30.assembled.split2.maxee.fasta \
-fastq_maxee 1
# Using Roger Edgar's max expected error filtering on our split fastq file

In [ ]:
!cat ../../SeqData/pear_merged-2015-06-30.assembled.split1.maxee.fasta \
../../SeqData/pear_merged-2015-06-30.assembled.split2.maxee.fasta > \
../../SeqData/pear_merged_2015_06_30.assembled.maxee.fasta
# Joining the files back together into one fasta file.

###Now we examine and cut the sequences using mothur

In [ ]:
%%mothur
summary.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.fasta)

Okay, so we have about 2M sequences after demultiplexing and the maxee control. There are no ambiguous bases. Most are in the correct size range, actually.
We will proceed through standard mothur operating procedure. (First testing it with a cropped dataset)

In [ ]:
!head -250000 ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.fasta > ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.crop.fasta

In [ ]:
!wc -l ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.fasta

In [ ]:
%%mothur
unique.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.fasta)

6mers 63s
7mers 55s
8mers 56s, 53s (61s with larger pcr db, 68s with even larger, 60s with medium, 68s with current, 69s with filtered current)
9mers 52s, 54s
10mers 62s
8 and 9mers seem comparable - just use 8 because good AND standard.

For filtered wider PCR database,
6mers 81s
7mers 70s
8mers 69s
9mers 62s, 61s <- we shall use this one.
10mers 75s

In [ ]:
%%mothur
align.seqs(candidate=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.unique.fasta, template=../../SeqData/db/silva.total.filter.pcr.filter.fasta, processors=4, flip=T, ksize=9)

^ Started at 3:52 PM finished at 4:12 PM

In [ ]:
%%mothur
summary.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.unique.align, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.names, processors=4)

In [ ]:
%%mothur
screen.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.unique.align, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.names, start=84, end=3899, minlength=298, maxlength=305, maxhomop=8, processors=4)

In [ ]:
%%mothur
summary.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.unique.good.align, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.good.names, processors=4)

In [ ]:
%%mothur
filter.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.unique.good.align, vertical=T, trump=., processors=4)

This filter step make sure that the sequences are all comparable. Now they start around the same place, and will end around the same place, so they will be comparable.

In [ ]:
%%mothur
summary.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.unique.good.filter.fasta, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.good.names, processors=4)

In [ ]:
%%mothur
screen.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.unique.good.filter.fasta, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.good.names, start=2, end=612, minlength=289, maxhomop=8, processors=4)

In [ ]:
%%mothur
summary.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.unique.good.filter.good.fasta, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.good.good.names, processors=4)

In [ ]:
%%mothur
deunique.seqs(fasta=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.unique.good.filter.good.fasta, name=../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.good.good.names)

In [ ]:
# FinalQC.fasta will be our last file.
# Here we are taking out all the alignment characters (-,.,etc.)
nprocs=4
!sed '/>/! s/-//g;/>/! s/\.//g' ../../SeqData/pear_merged_2015_07_03.assembled.demult.maxee.unique.good.filter.good.redundant.fasta > ../../SeqData/16SfinalQC.crop.fasta

In [ ]:
!head -100 ../../SeqData/16SfinalQC.fasta

In [ ]:
%%mothur
summary.seqs(fasta=../../SeqData/16SfinalQC.fasta, processors=4)

###Here we made the databases from mothur's reference Silva database. We took eukaryotic, archaeal, and bacterial sequences that should have hit our primers, combine them, cut them to our range, and filter out gaps.

In [ ]:
!mkdir ../../SeqData/db/
!curl -o ../../SeqData/db/silva_B.zip http://www.mothur.org/w/images/9/98/Silva.bacteria.zip
!curl -o ../../SeqData/db/silva_E.zip http://www.mothur.org/w/images/1/1a/Silva.eukarya.zip
!curl -o ../../SeqData/db/silva_A.zip http://www.mothur.org/w/images/3/3c/Silva.archaea.zip
!unzip ../../SeqData/db/silva_B.zip
!unzip ../../SeqData/db/silva_E.zip
!unzip ../../SeqData/db/silva_A.zip
# Getting the Silva databases that mothur recommends
# We aren't using these for our taxonomy assignments, per se - just to align our sequences for QC

In [ ]:
!cat silva.bacteria/silva.bacteria.fasta \
    silva.eukarya/silva.eukarya.fasta \
    Silva.archaea/silva.archaea.fasta \
    > ../../SeqData/db/silva.total.fasta
# Joining the ref files together

In [ ]:
%%mothur
filter.seqs(vertical=t, fasta=../../SeqData/db/silva.total.fasta, processors=4)
# Just gets rid of vertical gaps in alignment in the database - will speed things up later on.

In [ ]:
%%mothur
summary.seqs(fasta=../../SeqData/db/silva.total.filter.fasta, processors=4)

Trimming the sequences in the database - our primers actually trim it back to about just the bacterial database (14868 seqs/18491). We might try keeping the non-primer matched sequences in there, but just using the bacterial database (14956). 

Actually, there must be some archeal or whatever in there, because trimming the bacterial db using pcr.seqs cuts it down to 13615 sequences. So, might as well go with the full, filtered db.

In [ ]:
%%mothur
pcr.seqs(fasta=../../SeqData/db/silva.total.filter.fasta, oligos=../../SeqData/primers.txt, processors=4)

In [ ]:
%%mothur
summary.seqs(fasta=../../SeqData/db/silva.total.filter.pcr.fasta, processors=4)

PCRing the reference db with our sequences yielded only 250bp or so sequences, but ours are longer... I can try to expand the db by cutting at, say, 3600 and 7500 and see if that makes a difference - keeping still the smaller db for faster alignment.

It is better - come out more with 293bp seqs, not 250bp. I played around expanding or contracting it to optimize time/size.

In [ ]:
%%mothur
pcr.seqs(fasta=../../SeqData/db/silva.total.filter.fasta, start=3200, end=7200, processors=4)

In [ ]:
%%mothur
summary.seqs(fasta=../../SeqData/db/silva.total.filter.pcr.fasta, processors=4)

In [ ]:
%%mothur
filter.seqs(fasta=../../SeqData/db/silva.total.filter.pcr.fasta)